# Import everything here

In [22]:
from data_module import *
from RungeKutta import *

%reload_ext autoreload
%autoreload 2

# Define notebook parameters

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

alpha = 1.3
delta_t = 0.5
t_eval = np.arange(0, 5, delta_t)
num_samples_in_trajectory = math.floor(t_eval[-1] / delta_t)

save_location = "./figures/alpha_1,3_"

# Setup Tensorboard

In [24]:
%load_ext tensorboard
%tensorboard --logdir runge-kutta --port 6006

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Launching TensorBoard...

# Create dataset

In [25]:
dataset = create_dataset(alpha, t_eval, 50)

# Reshape dataset

In [26]:
dataset = reshape_dataset(dataset)

# Split dataset

In [27]:
training_set, validation_set, test_set = split_dataset(dataset, 0.6, 0.2, 0.2)

# Create NN model and define hyperparameters

In [28]:
hparams = {
    "hidden_layer_1": 512,
    "hidden_layer_2": 512,
    "hidden_layer_3": 512,
    "delta_t": delta_t,
    "batch_size": 8,
    "learning_rate": 1e-3,
    "num_workers": 8,
}

In [29]:
model = RungeKutta(hparams)

# Original Data

In [30]:
t_eval = np.delete(t_eval, -1, 0)

In [35]:
plot_trajectory(validation_set[:num_samples_in_trajectory, 0], t_eval, True, "./figures/v0_rk_alpha_1,3_original_data.png")

AttributeError: module 'matplotlib' has no attribute 'path'

<Figure size 720x720 with 0 Axes>

# Validation set

In [33]:
print("Score of the Model before training:", evaluate_model(model, validation_set, hparams, device))
recreation = recreate_trajectory(model, validation_set[0, 0, :], t_eval[0], t_eval[-1], delta_t, device)
plot_trajectory(recreation, t_eval, True, "./figures/v0_rk_alpha_1,3_b_train.png")

Score of the Model before training: 2.922951187667379


AttributeError: module 'matplotlib' has no attribute 'path'

<Figure size 720x720 with 0 Axes>

In [ ]:
train_dataloader = DataLoader(training_set,
                              shuffle=False,
                              batch_size=hparams["batch_size"],
                              num_workers=8)
validation_dataloader = DataLoader(validation_set,
                                    shuffle=False,
                                    batch_size=hparams["batch_size"],
                                    num_workers=8)

In [ ]:
trainer = pl.Trainer(
    max_epochs=100,
    log_every_n_steps=5,
    gpus=1 if torch.cuda.is_available() else None
)

trainer.fit(model, train_dataloader)

In [ ]:
model.hparams["delta_t"] = 0.05
t_eval_tmp = np.arange(0, 5, 0.05).round(2)
t_eval_tmp = np.delete(t_eval_tmp, -1, 0)
print("Score of the Model after training:", evaluate_model(model, validation_set, hparams, device))
recreation = recreate_trajectory(model, validation_set[0, 0, :], t_eval_tmp[0], t_eval_tmp[-1], model.hparams["delta_t"], device)
plot_trajectory(recreation, t_eval_tmp, True, "./figures/v0_rk_alpha_1,3_a_train.png")

In [ ]:
compute_and_plot_phase_portrait(model, 0.05, True, "./figures/v0_rk_alpha_1,3_phase_portrait.png")